In [4]:
import pyodbc
import pandas as pd
import numpy as np
import logging
import fnmatch
import os

server = 'IN3508853W1\SQLEXPRESS'
database = 'final_project'
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = cnxn.cursor()

lst = fnmatch.filter(os.listdir(r"C:\finalProjectFiles"), '*.csv')

temp_lst = [i[0:i.find('.csv')] for i in lst] #will store a list with all the table names without csv or df

for v in temp_lst:
    dir = f'C:\\finalProjectFiles\\{v}.csv';
    globals()[f'{v}_df'] = pd.read_csv(dir,quotechar="\"")
    globals()[f'{v}_df'].columns = globals()[f'{v}_df'].columns.astype(str)
    globals()[f'{v}_df'].replace({np.inf: np.nan, -np.inf: np.nan}, inplace=True)
    globals()[f'{v}_df'] = globals()[f'{v}_df'].fillna(0)


populateLandingTables(temp_lst, cursor, cnxn)



starting with  Categories
truncated Categories
CategoryID, CategoryName, Description
[1, 'Beverages', 'Soft drinks, coffees, teas, beers, and ales']
1, 'Beverages', 'Soft drinks, coffees, teas, beers, and ales'
INSERT INTO final_project.north.Categories(CategoryID, CategoryName, Description) values([1, 'Beverages', 'Soft drinks, coffees, teas, beers, and ales'])
[2, 'Condiments', 'Sweet and savory sauces, relishes, spreads, and seasonings']
2, 'Condiments', 'Sweet and savory sauces, relishes, spreads, and seasonings'
INSERT INTO final_project.north.Categories(CategoryID, CategoryName, Description) values([2, 'Condiments', 'Sweet and savory sauces, relishes, spreads, and seasonings'])
[3, 'Confections', 'Desserts, candies, and sweet breads']
3, 'Confections', 'Desserts, candies, and sweet breads'
INSERT INTO final_project.north.Categories(CategoryID, CategoryName, Description) values([3, 'Confections', 'Desserts, candies, and sweet breads'])
[4, 'Dairy Products', 'Cheeses']
4, 'Dairy Pr

ProgrammingError: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'Bon app''. (207) (SQLExecDirectW)")

In [3]:
def populateLandingTables(df_list,cursor, cnxn):
    for i in df_list:
        print('starting with ',i)
        df = globals()[f'{i}_df']
        cols = list(df.columns)
        cursor.execute(f"Truncate table north.{i}")
        print('truncated', i)

        
        columns_part = ', '.join(cols)
        print(columns_part)
        for index, row in df.iterrows():
            row_values = [row[col] for col in cols]
            row_valuesStr = r', '.join([repr(item) if isinstance(item, str) else str(item) for item in row_values])
            print(row_values)
            print(row_valuesStr)
            print(f"INSERT INTO final_project.north.{i}({columns_part}) values({row_values})")
            cursor.execute(f"INSERT INTO final_project.north.{i}({columns_part}) values({row_valuesStr})")
            cnxn.commit()
        print("done populating ",i)


